In [233]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk import word_tokenize
from tensorflow.keras.callbacks import EarlyStopping

# New Section

# New Section

In [234]:
df_pro = pd.read_csv('PromoterSequence.txt')

df_pro.head()

,>EP 1 (+) mt:CoI_1; range -100 to 200.
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...
1,>EP 2 (+) mt:CoIII_1; range -100 to 200.
2,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...
3,>EP 3 (+) plexB_1; range -100 to 200.
4,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...


In [235]:
df_pro.rename(columns ={'>EP 1 (+) mt:CoI_1; range -100 to 200.': 'Sequence'}, inplace =True)
df_pro.head()

,Sequence
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...
1,>EP 2 (+) mt:CoIII_1; range -100 to 200.
2,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...
3,>EP 3 (+) plexB_1; range -100 to 200.
4,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...


In [236]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [237]:
df_pro['is_promoter'] = 1
df_pro.head()

,Sequence,is_promoter
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,>EP 2 (+) mt:CoIII_1; range -100 to 200.,1
2,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
3,>EP 3 (+) plexB_1; range -100 to 200.,1
4,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1


In [238]:
df_pro=df_pro[df_pro["Sequence"].str.contains("EP")==False]

In [239]:
df_pro.reset_index(drop =True)

,Sequence,is_promoter
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1
...,...,...
11295,CGACAAAGTTTGATCCATGTGCATTCTTGGCGCCTTATCGATAGCT...,1
11296,CATATCTACATCTCGCTTGCTCCTTCCCTTTCGCTGCGTGTGTGTG...,1
11297,ATACCGCGGAAGCGCAAAAGTACCAGAATTTCCCTGGTATCGCGCT...,1
11298,ATTATTCCGAATTCTTTTATCAGATTTAAATATGGGAAACACTTTA...,1


In [240]:
df_npro = pd.read_csv('NonPromoterSequence.txt')

df_npro.head()

,>EP 1 (+) mt:CoI_1; range -400 to -100.
0,TAATTACATTATTTTTTTATTTACGAATTTGTTATTCCGCTTTTAT...
1,>EP 2 (+) mt:CoIII_1; range -400 to -100.
2,ATTTTTACAAGAACAAGACATTTAACTTTAACTTTATCTTTAGCTT...
3,>EP 3 (+) plexB_1; range -400 to -100.
4,AGAGATAGGTGGGTCTGTAACACTCGAATCAAAAACAATATTAAGA...


In [241]:
df_npro['is_promoter'] = 0
df_npro.head()

,>EP 1 (+) mt:CoI_1; range -400 to -100.,is_promoter
0,TAATTACATTATTTTTTTATTTACGAATTTGTTATTCCGCTTTTAT...,0
1,>EP 2 (+) mt:CoIII_1; range -400 to -100.,0
2,ATTTTTACAAGAACAAGACATTTAACTTTAACTTTATCTTTAGCTT...,0
3,>EP 3 (+) plexB_1; range -400 to -100.,0
4,AGAGATAGGTGGGTCTGTAACACTCGAATCAAAAACAATATTAAGA...,0


In [242]:
df_npro.rename(columns ={'>EP 1 (+) mt:CoI_1; range -400 to -100.': 'Sequence'}, inplace =True)
df_npro.head()

,Sequence,is_promoter
0,TAATTACATTATTTTTTTATTTACGAATTTGTTATTCCGCTTTTAT...,0
1,>EP 2 (+) mt:CoIII_1; range -400 to -100.,0
2,ATTTTTACAAGAACAAGACATTTAACTTTAACTTTATCTTTAGCTT...,0
3,>EP 3 (+) plexB_1; range -400 to -100.,0
4,AGAGATAGGTGGGTCTGTAACACTCGAATCAAAAACAATATTAAGA...,0


In [243]:
df_npro=df_npro[df_npro["Sequence"].str.contains("EP")==False]

In [244]:
df_npro.reset_index(drop =True)

,Sequence,is_promoter
0,TAATTACATTATTTTTTTATTTACGAATTTGTTATTCCGCTTTTAT...,0
1,ATTTTTACAAGAACAAGACATTTAACTTTAACTTTATCTTTAGCTT...,0
2,AGAGATAGGTGGGTCTGTAACACTCGAATCAAAAACAATATTAAGA...,0
3,TATGTATATAGAGATAGGCGTTGCCAATAACTTTTGCGTTTTTTGC...,0
4,AGAAATAATAGCTAGAGCAAAAAACAGCTTAGAACGGCTGATGCTC...,0
...,...,...
11295,TGGTAAAAAATTGTACACCTAACTAGTGCCTTCATGTATACCACCA...,0
11296,AGTGCAACTGGAGCCGTGCCGTGACCCACAGAGATCGCCCACTCGA...,0
11297,GCATGGATTTCATATTATCTTAATCGACTTGCTTTTATAAAATAGG...,0
11298,GTGACCAGGTTTTGCTCTAATGCGAAGTACGGATTGGGTAGAGATA...,0


In [245]:
display(pd.concat([df_pro, df_npro], ignore_index=True))

,Sequence,is_promoter
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1
...,...,...
22595,TGGTAAAAAATTGTACACCTAACTAGTGCCTTCATGTATACCACCA...,0
22596,AGTGCAACTGGAGCCGTGCCGTGACCCACAGAGATCGCCCACTCGA...,0
22597,GCATGGATTTCATATTATCTTAATCGACTTGCTTTTATAAAATAGG...,0
22598,GTGACCAGGTTTTGCTCTAATGCGAAGTACGGATTGGGTAGAGATA...,0


In [246]:
df_all = pd.concat([df_pro, df_npro], ignore_index=True)
df_all.head()

,Sequence,is_promoter
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1


In [247]:
def two_mer(string, size =2):
    return [string[x:x+size] for x in range(len(string)- size + 1)]

def three_mer(string, size =3):
    return [string[x:x+size] for x in range(len(string)- size + 1)]


def four_mer(string, size=4):
    return [string[x:x+size] for x in range(len(string)- size + 1)]

def five_mer(string, size=5):
    return [string[x:x+size] for x in range(len(string)- size + 1)]

def six_mer(string, size =6):
    return [string[x:x+size] for x in range(len(string)- size + 1)]


In [248]:
df_all['two-mer'] = df_all['Sequence'].apply(two_mer)

df_all['three-mer'] = df_all['Sequence'].apply(three_mer)

df_all['four-mer'] = df_all['Sequence'].apply(four_mer)

df_all['five-mer'] = df_all['Sequence'].apply(five_mer)

df_all['six-mer'] = df_all['Sequence'].apply(six_mer)


In [249]:
df_all['two-mer'] = df_all['two-mer'].apply(" ".join)
df_all['three-mer'] = df_all['three-mer'].apply(" ".join)
df_all['four-mer'] = df_all['four-mer'].apply(" ".join)
df_all['five-mer'] = df_all['five-mer'].apply(" ".join)
df_all['six-mer'] = df_all['six-mer'].apply(" ".join)


In [250]:
df_all.head()

,Sequence,is_promoter,two-mer,three-mer,four-mer,five-mer,six-mer
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1,TT TA AA AT TT TT TG GT TC CC CT TT TA AT TT T...,TTA TAA AAT ATT TTT TTG TGT GTC TCC CCT CTT TT...,TTAA TAAT AATT ATTT TTTG TTGT TGTC GTCC TCCT C...,TTAAT TAATT AATTT ATTTG TTTGT TTGTC TGTCC GTCC...,TTAATT TAATTT AATTTG ATTTGT TTTGTC TTGTCC TGTC...
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1,AT TA AG GC CT TC CA AA AA AT TT TG GC CT TT T...,ATA TAG AGC GCT CTC TCA CAA AAA AAT ATT TTG TG...,ATAG TAGC AGCT GCTC CTCA TCAA CAAA AAAT AATT A...,ATAGC TAGCT AGCTC GCTCA CTCAA TCAAA CAAAT AAAT...,ATAGCT TAGCTC AGCTCA GCTCAA CTCAAA TCAAAT CAAA...
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1,AA AG GC CT TT TC CC CC CT TT TT TA AA AT TG G...,AAG AGC GCT CTT TTC TCC CCC CCT CTT TTT TTA TA...,AAGC AGCT GCTT CTTC TTCC TCCC CCCT CCTT CTTT T...,AAGCT AGCTT GCTTC CTTCC TTCCC TCCCT CCCTT CCTT...,AAGCTT AGCTTC GCTTCC CTTCCC TTCCCT TCCCTT CCCT...
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1,TA AT TG GT TA AG GA AA AT TC CT TG GT TA AC C...,TAT ATG TGT GTA TAG AGA GAA AAT ATC TCT CTG TG...,TATG ATGT TGTA GTAG TAGA AGAA GAAT AATC ATCT T...,TATGT ATGTA TGTAG GTAGA TAGAA AGAAT GAATC AATC...,TATGTA ATGTAG TGTAGA GTAGAA TAGAAT AGAATC GAAT...
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1,AC CA AT TA AT TT TA AC CT TG GC CA AT TA AC C...,ACA CAT ATA TAT ATT TTA TAC ACT CTG TGC GCA CA...,ACAT CATA ATAT TATT ATTA TTAC TACT ACTG CTGC T...,ACATA CATAT ATATT TATTA ATTAC TTACT TACTG ACTG...,ACATAT CATATT ATATTA TATTAC ATTACT TTACTG TACT...


In [251]:

#X = df_all['six-mer']
y= df_all['is_promoter']
X= df_all['three-mer']

In [252]:
X

0        TTA TAA AAT ATT TTT TTG TGT GTC TCC CCT CTT TT...
1        ATA TAG AGC GCT CTC TCA CAA AAA AAT ATT TTG TG...
2        AAG AGC GCT CTT TTC TCC CCC CCT CTT TTT TTA TA...
3        TAT ATG TGT GTA TAG AGA GAA AAT ATC TCT CTG TG...
4        ACA CAT ATA TAT ATT TTA TAC ACT CTG TGC GCA CA...
                               ...                        
22595    TGG GGT GTA TAA AAA AAA AAA AAA AAT ATT TTG TG...
22596    AGT GTG TGC GCA CAA AAC ACT CTG TGG GGA GAG AG...
22597    GCA CAT ATG TGG GGA GAT ATT TTT TTC TCA CAT AT...
22598    GTG TGA GAC ACC CCA CAG AGG GGT GTT TTT TTT TT...
22599    TCA CAT ATA TAT ATT TTG TGA GAA AAA AAA AAT AT...
Name: three-mer, Length: 22600, dtype: object

In [253]:
X_train, X_test, y_train, y_test =train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [254]:
X

0        TTA TAA AAT ATT TTT TTG TGT GTC TCC CCT CTT TT...
1        ATA TAG AGC GCT CTC TCA CAA AAA AAT ATT TTG TG...
2        AAG AGC GCT CTT TTC TCC CCC CCT CTT TTT TTA TA...
3        TAT ATG TGT GTA TAG AGA GAA AAT ATC TCT CTG TG...
4        ACA CAT ATA TAT ATT TTA TAC ACT CTG TGC GCA CA...
                               ...                        
22595    TGG GGT GTA TAA AAA AAA AAA AAA AAT ATT TTG TG...
22596    AGT GTG TGC GCA CAA AAC ACT CTG TGG GGA GAG AG...
22597    GCA CAT ATG TGG GGA GAT ATT TTT TTC TCA CAT AT...
22598    GTG TGA GAC ACC CCA CAG AGG GGT GTT TTT TTT TT...
22599    TCA CAT ATA TAT ATT TTG TGA GAA AAA AAA AAT AT...
Name: three-mer, Length: 22600, dtype: object

In [255]:
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(X_train)
len(tokenizer2.word_counts)

67

In [256]:
# Convert words to sequence of numbers
train_sq = tokenizer2.texts_to_sequences(X_train)
test_sq = tokenizer2.texts_to_sequences(X_test)

In [257]:
tokenizer2.word_index

{'aaa': 1,
 'aac': 12,
 'aag': 17,
 'aat': 3,
 'aca': 14,
 'acc': 53,
 'acg': 54,
 'act': 35,
 'aga': 31,
 'agc': 24,
 'agg': 61,
 'agt': 20,
 'ata': 6,
 'atc': 26,
 'atg': 28,
 'att': 4,
 'caa': 9,
 'cac': 40,
 'cag': 30,
 'cat': 23,
 'cca': 27,
 'ccc': 63,
 'ccg': 57,
 'cct': 62,
 'cga': 37,
 'cgc': 42,
 'cgg': 58,
 'cgt': 55,
 'cta': 50,
 'ctc': 52,
 'ctg': 36,
 'ctt': 21,
 'gaa': 11,
 'gac': 60,
 'gag': 46,
 'gat': 29,
 'gca': 18,
 'gcc': 41,
 'gcg': 44,
 'gct': 34,
 'gga': 47,
 'ggc': 48,
 'ggg': 64,
 'ggt': 56,
 'gta': 43,
 'gtc': 59,
 'gtg': 32,
 'gtt': 16,
 'nga': 67,
 'nng': 66,
 'nnn': 65,
 'taa': 5,
 'tac': 45,
 'tag': 51,
 'tat': 8,
 'tca': 22,
 'tcc': 49,
 'tcg': 38,
 'tct': 39,
 'tga': 25,
 'tgc': 19,
 'tgg': 33,
 'tgt': 15,
 'tta': 7,
 'ttc': 13,
 'ttg': 10,
 'ttt': 2}

In [258]:
max_length=max(len(x) for x in X_train) 
padding_type = 'post'
trunc_type = 'post'
embedding_dim = 200
vocab_size = 68

In [259]:
train_padded = pad_sequences(train_sq, maxlen=max_length, padding = padding_type, truncating=trunc_type)

test_padded = pad_sequences(test_sq, maxlen=max_length, padding = padding_type, truncating=trunc_type)

In [260]:
train_padded = pd.DataFrame(train_padded )

In [261]:
from tensorflow.keras.layers import Bidirectional

In [262]:
model_1 = Sequential()
model_1.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model_1.add(Conv1D(200, 5, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2))
model_1.add(Conv1D(200, 5, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2))
model_1.add(Flatten())
#model_1.add(Dense(10, activation = 'relu'))
#model_1.add(Dense(10, activation='relu'))
#model_1.add(LSTM(64))


#model_1.add(Bidirectional(LSTM(64,return_sequences=True)))
#model_1.add(Bidirectional(LSTM(32)))

model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model_1.summary()
es=EarlyStopping(monitor= 'val_accuracy', min_delta =0, patience=5, restore_best_weights=True)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 1195, 200)         13600     
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 1191, 200)         200200    
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 595, 200)          0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 591, 200)          200200    
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 295, 200)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 59000)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)               

In [263]:
model_1.fit(train_padded, y_train, validation_data = (test_padded, y_test), batch_size=256, epochs=30, callbacks=[es])

Epoch 1/30
71/71 [==============================] - 14s 188ms/step - loss: 0.4981 - accuracy: 0.7419 - val_loss: 0.3863 - val_accuracy: 0.8257
Epoch 2/30
71/71 [==============================] - 13s 188ms/step - loss: 0.3257 - accuracy: 0.8607 - val_loss: 0.3010 - val_accuracy: 0.8712
Epoch 3/30
71/71 [==============================] - 13s 190ms/step - loss: 0.2595 - accuracy: 0.8950 - val_loss: 0.2681 - val_accuracy: 0.8841
Epoch 4/30
71/71 [==============================] - 14s 191ms/step - loss: 0.2294 - accuracy: 0.9090 - val_loss: 0.2927 - val_accuracy: 0.8847
Epoch 5/30
71/71 [==============================] - 14s 192ms/step - loss: 0.2027 - accuracy: 0.9199 - val_loss: 0.2588 - val_accuracy: 0.8903
Epoch 6/30
71/71 [==============================] - 14s 193ms/step - loss: 0.1783 - accuracy: 0.9312 - val_loss: 0.2537 - val_accuracy: 0.8980
Epoch 7/30
71/71 [==============================] - 14s 194ms/step - loss: 0.1666 - accuracy: 0.9359 - val_loss: 0.2543 - val_accuracy: 0.8965

In [264]:
df_test = pd.read_csv('test.csv')
df_test.head()

,Sequence
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...


In [265]:
df_test['three-mer'] = df_test['Sequence'].apply(three_mer)
df_test['three-mer'] = df_test['three-mer'].apply(" ".join)

In [266]:
df_test.head()

,Sequence,three-mer
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,AAT ATC TCT CTC TCT CTG TGT GTT TTG TGT GTT TT...
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,TTC TCT CTC TCT CTA TAA AAG AGA GAA AAA AAG AG...
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,ACA CAT ATT TTT TTT TTA TAC ACG CGG GGT GTC TC...
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,CAC ACT CTG TGA GAA AAA AAA AAA AAA AAA AAA AA...
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,GGG GGA GAG AGT GTG TGG GGC GCA CAA AAC ACA CA...


In [267]:
test_s = tokenizer2.texts_to_sequences(df_test['three-mer'])
test_padded = pad_sequences(test_s, maxlen=max_length, padding = padding_type, truncating=trunc_type)

In [268]:
pred_1= model_1.predict(test_padded)
pred_1

array([[3.9647698e-02],
       [9.9806148e-01],
       [4.4842693e-01],
       [1.2538700e-03],
       [7.3162321e-04],
       [4.9390745e-01],
       [9.2509550e-01],
       [9.7372931e-01],
       [3.9397642e-02],
       [9.9985301e-01],
       [9.9962032e-01],
       [1.3438966e-02],
       [8.3039235e-03],
       [4.6776441e-01],
       [2.8488874e-01],
       [9.9172264e-01],
       [4.7297380e-03],
       [5.1160581e-02],
       [4.7579378e-01],
       [9.4207036e-01],
       [4.3073134e-03],
       [8.8587123e-01],
       [9.9318051e-01],
       [3.6702780e-03],
       [1.0815105e-01],
       [9.9775273e-01],
       [9.8283285e-01],
       [9.9883956e-01],
       [9.9457848e-01],
       [7.7237743e-01],
       [2.0413617e-02],
       [2.6753359e-03],
       [1.9616568e-01],
       [1.0990869e-01],
       [9.9946409e-01],
       [6.5998025e-03],
       [3.3273641e-02],
       [8.0470890e-01],
       [5.1946178e-02],
       [5.9076209e-02],
       [8.1272644e-01],
       [8.891478

In [269]:
eval = np.array(pred_1)
eval[eval>=0.5]=1
eval[eval<0.5]=0

In [270]:
pred_1 = pd.DataFrame(data = eval, columns=['final_pred'])
pred_1.head(10)

,final_pred
0,0.0
1,1.0
2,0.0
3,0.0
4,0.0
5,0.0
6,1.0
7,1.0
8,0.0
9,1.0


In [271]:
df_test['predictions']= pred_1
df_test.head()

,Sequence,three-mer,predictions
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,AAT ATC TCT CTC TCT CTG TGT GTT TTG TGT GTT TT...,0.0
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,TTC TCT CTC TCT CTA TAA AAG AGA GAA AAA AAG AG...,1.0
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,ACA CAT ATT TTT TTT TTA TAC ACG CGG GGT GTC TC...,0.0
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,CAC ACT CTG TGA GAA AAA AAA AAA AAA AAA AAA AA...,0.0
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,GGG GGA GAG AGT GTG TGG GGC GCA CAA AAC ACA CA...,0.0


In [272]:
submission_3 = df_test.drop(columns =['three-mer'])
submission_3.to_csv('submissions_3.csv', index=False)